In [1]:
from tflite_support.task import core
from tflite_support.task import processor
from tflite_support.task import vision
import pandas as pd
import shutil 
import cv2
import os
import duckdb
import sqlalchemy
import random
import xml.etree.ElementTree as ET
from xml.dom import minidom
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [2]:
## change the model path to attached model file
MODEL_PATH = "/home/abdullah/personal/github_projects/sm_city/result_renerator/model_arch_4.tflite"
## Change the path to dataset folder 
base_images = '/home/abdullah/personal/github_projects/efficientnet_lite_preprocess/dataset/images/'
## change the path to where you want to save the annotated images
annotated_images = '/home/abdullah/personal/github_projects/sm_city/auto_annotator/annotated_images/'
## change the path to where you want to save the annotations
annotations_folder = '/home/abdullah/personal/github_projects/sm_city/auto_annotator/annotations/'
if os.path.exists(annotated_images):
    shutil.rmtree(annotated_images)
    shutil.rmtree(annotations_folder)
    
os.makedirs(annotated_images)
os.makedirs(annotations_folder)




In [3]:
images = shutil.os.listdir(base_images)
images = random.sample(images, 200) # random sample (for testing)
images = [base_images + image for image in images] 


In [4]:
base_options = core.BaseOptions(
      file_name=MODEL_PATH, use_coral=False, num_threads=4)
detection_options = processor.DetectionOptions(
      max_results=3, score_threshold=.6)   
options = vision.ObjectDetectorOptions(
      base_options=base_options, detection_options=detection_options)  
detector = vision.ObjectDetector.create_from_options(options)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
# Function for inference 
def inference(image_array):
    input_tensor = vision.TensorImage.create_from_array(image_array)
    results = detector.detect(input_tensor)
    return results.detections

In [6]:
results = []
for image_path in images:
    image = cv2.imread(image_path)
    detections = inference(image)
    if detections:
        image_width = image.shape[1]
        image_height = image.shape[0]
        for detection in detections:
            result = {}
            result['image_path'] = image_path
            result['height'] = image_height
            result['width'] = image_width
            result['detection'] = detection
            results.append(result)   
    

In [7]:
results

[{'image_path': '/home/abdullah/personal/github_projects/efficientnet_lite_preprocess/dataset/images/2f769d2651a07247dff2d4190bfe09e8.jpg',
  'height': 1080,
  'width': 1920,
  'detection': Detection(bounding_box=BoundingBox(origin_x=1099, origin_y=263, width=524, height=193), categories=[Category(index=0, score=0.68359375, display_name='', category_name='GRAFFITI')])},
 {'image_path': '/home/abdullah/personal/github_projects/efficientnet_lite_preprocess/dataset/images/2f769d2651a07247dff2d4190bfe09e8.jpg',
  'height': 1080,
  'width': 1920,
  'detection': Detection(bounding_box=BoundingBox(origin_x=455, origin_y=225, width=932, height=245), categories=[Category(index=0, score=0.671875, display_name='', category_name='GRAFFITI')])},
 {'image_path': '/home/abdullah/personal/github_projects/efficientnet_lite_preprocess/dataset/images/afef5b55201890dbad860759a1cd41c4.jpg',
  'height': 1080,
  'width': 1920,
  'detection': Detection(bounding_box=BoundingBox(origin_x=721, origin_y=406, widt

In [8]:
def result_to_bascal_voc(results):
    for result in results:
        image_path = result['image_path']
        shutil.copy(image_path, annotated_images)
        image_name = image_path.split('/')[-1]
        image_name = image_name.split('.')[0]
        image_width = result['width']
        image_height = result['height']
        detection = result['detection']
        bounding_box = detection.bounding_box
        categories = detection.categories
        for category in categories:
            category_name = category.category_name
            category_score = category.score
            xmin = bounding_box.origin_x
            ymin = bounding_box.origin_y
            xmax = bounding_box.origin_x + bounding_box.width
            ymax = bounding_box.origin_y + bounding_box.height


            xml = ET.Element('annotation')
            ET.SubElement(xml, 'folder').text = 'images'
            ET.SubElement(xml, 'filename').text = image_name
            ET.SubElement(xml, 'path').text = image_path
            source = ET.SubElement(xml, 'source')
            ET.SubElement(source, 'database').text = 'Unknown'
            size = ET.SubElement(xml, 'size')
            ET.SubElement(size, 'width').text = str(image_width)
            ET.SubElement(size, 'height').text = str(image_height)
            ET.SubElement(size, 'depth').text = '3'
            ET.SubElement(xml, 'segmented').text = '0'
            object = ET.SubElement(xml, 'object')
            ET.SubElement(object, 'name').text = category_name
            ET.SubElement(object, 'pose').text = 'Unspecified'
            ET.SubElement(object, 'truncated').text = '0'
            ET.SubElement(object, 'difficult').text = '0'
            bndbox = ET.SubElement(object, 'bndbox')
            ET.SubElement(bndbox, 'xmin').text = str(xmin)
            ET.SubElement(bndbox, 'ymin').text = str(ymin)
            ET.SubElement(bndbox, 'xmax').text = str(xmax)
            ET.SubElement(bndbox, 'ymax').text = str(ymax)
            xmlstr = minidom.parseString(ET.tostring(xml)).toprettyxml(indent="   ")
            
            with open(f'{annotations_folder}{image_name}.xml', "w") as f:
                f.write(xmlstr)

In [9]:
# this function will write the annotations in the annotations folder and copy the images in the annotated_images folder

result_to_bascal_voc(results)